In [1]:
import preprocessing as preproc
from utils import *
import collections
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm


Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
class MetricsData(object):

    def __init__(self,word,appearances,metrics_dict):
        self.word = word
        self.appearances = appearances
        self.metrics_dict = metrics_dict
        

In [3]:
def status((groundTruthValue,predictedValue)):
    if(groundTruthValue):
        if(predictedValue):
            return "TP"
        return "FN"
    
    if(predictedValue):
        return "FP"
    
    return "TN"

def get_metrics(true_word_mask,predicted_word_mask):
    truth2predictedArr = zip(true_word_mask,predicted_word_mask)
    statusArr = [status(truth2predicted) for truth2predicted in truth2predictedArr]
    counter = collections.defaultdict(int,collections.Counter(statusArr).most_common())
    
    tp = counter['TP']
    fp = counter['FP']
    tn = counter['TN']
    fn = counter['FN']
    
    precision = float(tp)/max(1,(tp+fp))
    recall = float(tp)/max(1,(tp+fn))

    return {"precision":precision,"recall":recall}

def containsWord(sentence, word):
    return word in sentence

def get_word_mask(captions,word):
    return [containsWord(caption, word) for caption in captions]

def computeMetricsData(word):
    true_word_mask = get_word_mask(true_captions,metric_word)
    predicted_word_mask = get_word_mask(predicted_captions,metric_word)

    nr_instances = sum(true_word_mask)
    metrics_dict = get_metrics(true_word_mask,predicted_word_mask)

    return MetricsData(metric_word,nr_instances,metrics_dict)

def displayMetrics(metric_data_arr):
    
    for metric_data in metric_data_arr:
       
        print(metric_data.word+" => "+str(metric_data.appearances)+" instances")
        print("\tPrecision: "+str(metric_data.metrics_dict["precision"]))
        print("\tRecall: "+str(metric_data.metrics_dict["recall"]))
        print("")

def displayAggMetrics(metric_data_arr):
    total_instances = sum([metric_data.appearances for metric_data in metric_data_arr])
    
    for key in metric_data_arr[0].metrics_dict:
        metric_sum = sum([metric_data.appearances*metric_data.metrics_dict[key] for metric_data in metric_data_arr])        
        weighted_metric = float(metric_sum)/total_instances
        print("Weighted "+str(key)+str(": ") + str(weighted_metric))
        
def bleu_score_metric(reality, prediction):
    return nltk.translate.bleu_score.sentence_bleu([reality], prediction)

def compute_bleu_score_metric(predicted_captions, real_captions):
    real_captions = reduce(list.__add__, [list(real_caption) for real_caption in tqdm(real_captions)])
    bleu_scores = [bleu_score_metric(real_captions[i], predicted_captions[i]) for i in tqdm(range(len(predicted_captions)))]
    return np.average(bleu_scores)

def get_nr_repetitions(predicted_caption):
    counter = preproc.most_common_words([predicted_caption])
    repetition_values = [nr_app for (word,nr_app) in counter if nr_app > 1]
    return sum(repetition_values) / 2 

# Load Data

In [ ]:
base_path = app_3_length_15_data_path

train_path = base_path + train_folder
val_path = base_path + val_folder

test_indexed_captions_path = base_path + val_folder + batch_folder + indexed_captions_folder
test_raw_captions_path = base_path + val_folder + batch_folder+captions_folder

specific_captions = "app_3_length_15_past_word_70_epoch_predicted_captions"

In [ ]:
(test_captions_raw,_) = preproc.get_captions_raw_and_indexed(test_raw_captions_path,test_indexed_captions_path)
true_captions = [list(test_caption_raw)[0] for test_caption_raw in test_captions_raw]

max_size = len(true_captions)
predicted_captions = preproc.load_obj(val_path+predictions_folder+specific_captions)[:max_size]


print(len(true_captions))
print(len(predicted_captions))


# Specific words metrics

In [ ]:
metric_words = ['tennis','snow','train','cat','red','yellow','food','kitchen']
metric_data_arr = [computeMetricsData(metric_word) for metric_word in tqdm(metric_words)]

In [ ]:
displayMetrics(metric_data_arr)

In [ ]:
displayAggMetrics(metric_data_arr)

# All words metrics

In [ ]:
all_words = dict(preproc.most_common_words(true_captions)).keys()
all_metric_data_arr = [computeMetricsData(metric_word) for metric_word in tqdm(all_words)]
displayAggMetrics(all_metric_data_arr)

# BLEU Metric

In [ ]:
compute_bleu_score_metric(predicted_captions,true_captions)

# Repetition Metric

In [ ]:
total_nr_repetitions = sum([get_nr_repetitions(predicted_caption) for predicted_caption in tqdm(predicted_captions)])
total_nr_words = sum(len(caption.split()) for caption in tqdm(predicted_captions))
repetition_score = float(total_nr_repetitions) / total_nr_words

print("Total nr repetitions = "+str(total_nr_repetitions))
print("Total nr words = "+str(total_nr_words))
print("Repetition score = "+str(repetition_score))


In [7]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, MaxoutDense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,Input,merge,UpSampling2D

from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
import numpy as np

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL.Image as Image

import json
from tqdm import tqdm

from keras.optimizers import SGD, RMSprop, Adam

from utils import *
import cPickle as pickle
from matplotlib import pyplot as plt

from itertools import compress

import shutil
import string

import collections

import matplotlib.patches as patches

import os

from keras import callbacks

ImportError: No module named tqdm

In [5]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu',border_mode='same', input_shape=(1, 9, 9)))
model.add(Convolution2D(32, 3, 3, activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

model.compile(optimizer=Adam(lr=0.001),
                loss='mse')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 32, 9, 9)      320         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 32, 9, 9)      9248        convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 32, 4, 4)      0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 64, 4, 4)      18496       maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [26]:
inputs = Input((1, 9, 9))
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
# conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
# pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
# conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
# pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool2)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

up8 = merge([UpSampling2D(size=(2, 2))(conv5), conv2], mode='concat', concat_axis=1)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv5], mode='concat', concat_axis=1)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv8)

model = Model(input=inputs, output=conv10)

model.summary()

ValueError: "concat" mode can only merge layers with matching output shapes except for the concat axis. Layer shapes: [(None, 64, 8, 8), (None, 512, 2, 2)]

In [33]:
inputs = Input((1, 9, 9))
conv1 = Convolution2D(32, 3, 3, activation='relu')(inputs)
conv1 = Convolution2D(32, 3, 3, activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# conv2 = Convolution2D(64, 3, 3, activation='relu')(pool1)
# conv2 = Convolution2D(64, 3, 3, activation='relu')(conv2)
# pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
# conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
# pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
# conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
# pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool1)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

# up8 = merge([UpSampling2D(size=(2, 2))(conv5), conv2], mode='concat', concat_axis=1)
# conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
# conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up9 = merge([UpSampling2D(size=(2, 2))(conv5), conv1], mode='concat', concat_axis=1)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv5)

model = Model(input=inputs, output=conv10)

model.summary()

ValueError: "concat" mode can only merge layers with matching output shapes except for the concat axis. Layer shapes: [(None, 512, 4, 4), (None, 32, 5, 5)]